In [21]:
import pandas as pd
from math import cos, pi, atan, floor, sqrt

# Load/Set base data

In [22]:
# TODO: Link the decans file from drive or somehow attach it. Probably host on github and link that https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92
# url = 'copied_raw_GH_link'
# df1 = pd.read_csv(url)
df = pd.read_csv('decans.csv')

# TODO: Create spell list as CSV and load that as well

zodiac_order = {'Aries': 1, 'Taurus': 2, 'Gemini': 3, 'Cancer': 4, 'Leo': 5, 'Virgo': 6, 'Libra': 7, 'Scorpio': 8,
                'Sagittarius': 9, 'Capricorn': 10, 'Aquarius': 11, 'Pisces': 12}

In [23]:
df.index = df['Zodiacal Position'].str.split(' ').apply(
    lambda x: (int(zodiac_order[x[0]])-1)*3 + int(x[1][0]) -1).to_list()
df = df.sort_index()
df['decan_position'] = df.index
# df["zodiac_max"] = 10*df.index
# df["zodiac_min"] = df["zodiac_max"]-10
df

,Decan,Keyword,Zodiacal Position,Planet,Sephirah,Page,Influence,Places,Materials,Spell,decan_position
0,Ruax,Mind,Aries 1st,Mars,Chokmah,70,"Will, perception, sanity, drunkenness, hotter ...","Television studios or showrooms, hypnosis clin...","Cathode-ray tubes, spirals, poppies, down- war...",NaN,0
1,Barsafael,What lies behind,Aries 2nd,Sun,Binah,62,"Hidden truth, subconscious mind, primal unconf...","Deep beneath the ocean or in hidden caves, cit...","Ambergris, petroleum, white noise, artesian wa...",NaN,1
2,Arôtosael,Making and Breaking,Aries 3rd,Jupiter,Chesed,61,The cycle of making and destroying in all its ...,"Decisive battlefields, particle accelerators, ...",Ram’s horn (and anything else in a Fibonacci s...,Dust to Dust (Unraveling),2
3,Sahu,Wilds,Taurus 1st,Venus,Geburah,70,"Concepts and activities such as archery, hunti...","Mountainsides, the walls of Mycenae and Machu ...","Basalt, sequoia wood, subsonics, arrowheads, s...",Find the Path (Knowing),3
4,Iudal,Gate,Taurus 2nd,Mercury,Tiphareth,67,"Boundaries, roads, portals, rivers, bridges, a...","Bridges, crossroads, doorways (especially post...","Keys, two-headed coins, juniper, alexandrites,...",Teleport (Patterning),4
5,Ouare,Technology,Taurus 3rd,Saturn,Netzach,69,"Constructs, machines, alloys, and vehicles. So...","Garages, metallurgical laboratories, inside mo...","Gears and clockwork, plastic, microchips, bras...",NaN,5
6,Sphandôr,Knowledge,Gemini 1st,Mercury,Hod,71,"The sensorium, intellect, passive knowledge, i...","Computer rooms, hexagonal spaces, paintings or...","Agates and cat’s eyes, musk, hexagons, fennel,...",Something for my research,6
7,Belbel,Pain,Gemini 2nd,Venus,Yesod,63,"Pain, torment, sharpness, shadow, shock, loss,...","Torture chambers, death camps, deep in the rai...","Funeral crepe, surgical steel, rust, ammonia, ...",(Shielding),7
8,Kurtaêl,Necromancy,Gemini 3rd,Saturn,Malkuth,68,"Death, fear, decay and disease, the undead.","Graveyards, terminal wards of hospitals, ruine...","Rowan, cypress, myrrh, contaminated syringes, ...",NaN,8
9,Methiax,Illusion and Creation,Cancer 1st,Moon,Chokmah,68,"True creation, form, shapes reality and dreams...","Artist’s studios, movie theaters and movie set...","Pyrites, zircons, flashbulbs, eggs, pomegranat...",NaN,9


# The Secrets

In [24]:
# Adjustable params
time = 19640213123000 / 86400000

In [25]:
# TODO: Add params according to colab docs so this field can be hidden after the params are set
# See https://colab.research.google.com/notebooks/forms.ipynb#scrollTo=3jKM6GfzlgpS

planet_orbital_cycles = {
    "Moon": 30.0,
    "Mercury": 88.0,
    "Venus": 224.7,
    "Mars": 687.0,
    "Jupiter": 4331.0,
    "Saturn": 10747.0,
    "Sun": 365.25
}

opposing_planets = {
    "Sun":"Moon",
    "Moon":"Sun",
    "Mercury":None,
    "Venus":"Mars",
    "Mars":"Venus",
    "Jupiter":"Saturn",
    "Saturn":"Jupiter"
}

def get_planet_coordinates(time: int, planet: str, sun_coordinates):
    return {"x": .3 * cos(time/planet_orbital_cycles[planet]) + sun_coordinates["x"],
            "y": .32 * cos(time/(planet_orbital_cycles[planet]+.14*pi)) + sun_coordinates["y"],
            "z": .1 * cos(time/(planet_orbital_cycles[planet]/182.5))+ sun_coordinates["z"]}

sun_coordinates = get_planet_coordinates(time, "Sun", {"x":0,"y":0,"z":0})
sun_coordinates_future = get_planet_coordinates(time + 21_600_000, "Sun", {"x":0,"y":0,"z":0})

In [26]:
def generate_mana_level(row):
    # Base mana level
    mana_level = 0

    # Generate planet mana value
    mana_level = add_planet_mana(mana_level, row['Planet'])

    if opposing_planets[row['Planet']]:
        mana_level = add_planet_mana(mana_level, opposing_planets[row['Planet']])

    # check zodiac
    mana_level = add_zodiac_mana(row['decan_position'], mana_level)

    return mana_level


def add_zodiac_mana(decan_position, mana_level):
    zodiac_degrees = atan(sun_coordinates["y"] / sun_coordinates["x"]) * 360 / (2 * pi)
    zodiac_pos_tens = floor(zodiac_degrees / 10)
    zodiac_pos_thirty = floor(zodiac_degrees / 30)
    if zodiac_pos_tens == decan_position:
        mana_level += 1
    if zodiac_pos_thirty == floor(decan_position / 3):
        mana_level += 1

    opposing_decan_position = (decan_position + 18) % 36
    if zodiac_pos_tens == opposing_decan_position:
        mana_level -= 1
    if zodiac_pos_thirty == floor(opposing_decan_position / 3):
        mana_level -= 1
    return mana_level


def add_planet_mana(mana_level, planet):
    coords_current = get_planet_coordinates(time, planet, sun_coordinates)
    coords_future = get_planet_coordinates(time + 21_600, planet, sun_coordinates)
    planet_mana = 0
    if coords_current["z"] >= 0:
        cur_dir = get_distance(coords_current)
        fut_dir = get_distance(coords_future)
        planet_mana =  cur_dir - fut_dir
        planet_mana = -1 if planet_mana > 0 else 1 if planet_mana < 0 else 0
    print(f"{planet} has {planet_mana}")
    mana_level += planet_mana
    return mana_level


def get_distance(coords):
    return sqrt(coords["x"] ** 2 + coords["y"] ** 2 + coords["z"] ** 2)

# Generate Mana Levels

In [27]:
temp = df.apply(generate_mana_level,axis=1)
df.insert(2, 'Mana', temp)
df = df.sort_values(by=['Mana','Decan'],ascending=False)

Mars has 1
Venus has 1
Sun has -1
Moon has -1
Jupiter has 0
Saturn has 1
Venus has 1
Mars has 1
Mercury has -1
Saturn has 1
Jupiter has 0
Mercury has -1
Venus has 1
Mars has 1
Saturn has 1
Jupiter has 0
Moon has -1
Sun has -1
Mars has 1
Venus has 1
Venus has 1
Mars has 1
Sun has -1
Moon has -1
Jupiter has 0
Saturn has 1
Mars has 1
Venus has 1
Mercury has -1
Saturn has 1
Jupiter has 0
Venus has 1
Mars has 1
Venus has 1
Mars has 1
Saturn has 1
Jupiter has 0
Mercury has -1
Mars has 1
Venus has 1
Jupiter has 0
Saturn has 1
Moon has -1
Sun has -1
Jupiter has 0
Saturn has 1
Mars has 1
Venus has 1
Sun has -1
Moon has -1
Saturn has 1
Jupiter has 0
Venus has 1
Mars has 1
Mercury has -1
Saturn has 1
Jupiter has 0
Jupiter has 0
Saturn has 1
Venus has 1
Mars has 1
Jupiter has 0
Saturn has 1
Moon has -1
Sun has -1
Mars has 1
Venus has 1


In [28]:
df

,Decan,Keyword,Mana,Zodiacal Position,Planet,Sephirah,Page,Influence,Places,Materials,Spell,decan_position
3,Sahu,Wilds,3,Taurus 1st,Venus,Geburah,70,"Concepts and activities such as archery, hunti...","Mountainsides, the walls of Mycenae and Machu ...","Basalt, sequoia wood, subsonics, arrowheads, s...",Find the Path (Knowing),3
11,Saphathoraél,Water,2,Cancer 3rd,Venus,Chesed,71,"Water, liquids and their conceptual counterpar...","Oceans, lakes, ships, reservoirs, wells and oa...","Fish, the blood of drowned men, aquamarines an...",NaN,11
0,Ruax,Mind,2,Aries 1st,Mars,Chokmah,70,"Will, perception, sanity, drunkenness, hotter ...","Television studios or showrooms, hypnosis clin...","Cathode-ray tubes, spirals, poppies, down- war...",NaN,0
14,Roêlêd,Protection and Warning,2,Leo 3rd,Mars,Netzach,70,"Strong walls, thorns, antibiotics, defense, gu...",Anywhere under surveillance or an electric-eye...,"Owl or raven feathers, oxhide, “M” shapes, dra...",NaN,14
5,Ouare,Technology,2,Taurus 3rd,Saturn,Netzach,69,"Constructs, machines, alloys, and vehicles. So...","Garages, metallurgical laboratories, inside mo...","Gears and clockwork, plastic, microchips, bras...",NaN,5
18,Naôth,Communication and Empathy,2,Libra 1st,Venus,Chokmah,68,"Communication, language, imagery, symbolism, a...","Radio stations, printers, shared telepathic ex...","Vervain, bay leaves, “V” shapes, stringed inst...",Tongues (Knowing?),18
25,Eneuth,Fire,2,Sagittarius 2nd,Mars,Yesod,63,Fire and it's metaphorical equivalents: inspir...,"Bonfires, firestorms, tropical or subtropical ...","Obsidian, asbestos, sulfur, fire opals, sodium...",Fireball (Fraying),25
10,Charchnoumis,Animal,2,Cancer 2nd,Mars,Binah,63,"Animals, bestial thoughts and instincts, shape...","Zoos, slaughterhouses, nature preserves, fores...","Fresh animals’ blood or skin, bloodstones, fre...","Hear me, my Kin (Ruling)",10
17,Buldumêch,Wealth,2,Virgo 3rd,Venus,Malkuth,63,"Wealth, treasure, gold, finance, greed.","Banks, stock market floors, gold mines, custom...","Money, anything expensive and hoarded, jackdaw...",All that Glitters (Knowing),17
35,Bianakith,Body,2,Pisces 3rd,Mars,Malkuth,63,"The human body and mind, the template of human...","Crowded cities, human-scale statuary, portrait...","Lungwort, liverwort, elm, river clay, human bl...",Goliath (Perfecting),35


In [ ]:
def test(decan_series):
    print(decan_series)

# Output